In [ ]:
import stereo as st
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np

Bin = 20
SN = 'C1'
name = SN + '_RB'
path = '/data/work/output/Stereopy/Clustering/' + SN + '/' + name + '/Bin_' + str(Bin) + '/'
outdir = '/data/work/output/Trajectory/' + SN + '/' + name + '/Bin_' + str(Bin) + '/'
os.makedirs(outdir, exist_ok=True)

data = st.io.read_h5ad(file_path = path + name + '_Bin' + str(Bin) + '.h5ad',  flavor='stereopy')

data.tl.paga(groups='leiden', neighbors_key='neighbors')

data.tl.result['paga']

data.plt.interact_cluster(res_key='leiden')

data.plt.paga_plot(
    adjacency='connectivities_tree', 
    threshold=0.01, 
    layout='fr', 
    out_path =  outdir + name + '_paga_plot.pdf',
    out_dpi = 300  
    )

data.plt.draw_graph(
        adjacency='connectivities_tree',
        threshold=0.01,
        layout='fa',
        out_path =  outdir + name + '_paga_draw_graph.pdf',
        out_dpi = 300  
        )

data.plt.paga_compare(
        adjacency='connectivities_tree',
        threshold=0.01,
        out_path =  outdir + name + '_paga_compare_tree.pdf',
        out_dpi = 300  
        )

data.plt.paga_compare(
        adjacency='connectivities',
        threshold=0.01,
        out_path =  outdir + name + '_paga_compare.pdf',
        out_dpi = 300  
        )

x_raw = data.position[:, 0]
y_raw = data.position[:, 1]

data.plt.plot_cluster_traj(
        data.tl.result['paga']['connectivities_tree'].todense(),
        x_raw,
        y_raw,
        data.cells['leiden'].to_numpy(),
        lower_thresh_not_equal=0.95,
        count_thresh=100,
        eps_co=30,
        check_surr_co=20,
        show_ticks=True,
        type_traj="straight",
        plotting_scale_width=400,
        out_path =  outdir + name + '_paga_cluster_traj.pdf',
        out_dpi = 300 
        )

# setting the `iroot` to data.tl.result
data.tl.result['iroot'] = np.flatnonzero(data.cells['leiden'] == '1')[100]  # 'Heart'
data.tl.dpt(n_branchings=0)

data.plt.plot_vec(
        data.position[:, 0] - data.position[:, 0].min(),
        data.position[:, 1] - data.position[:, 1].min(),
        data.cells['leiden'].to_numpy(),
        data.tl.result['dpt_pseudotime'],
        type='stream',
        line_width=0.5,
        background='field',
        num_pix=50, 
        filter_type='gauss',
        sigma_val=1,
        radius_val=3,
        density=2,
        seed_val=0,
        num_legend_per_col=20,
        dpi_val=1000,
        out_path =  outdir + name + '_dpt_stram_field.svg'
        )

data.plt.plot_vec(
        data.position[:, 0] - data.position[:, 0].min(),
        data.position[:, 1] - data.position[:, 1].min(),
        data.cells['leiden'].to_numpy(),
        data.tl.result['dpt_pseudotime'],
        type='stream',
        line_width=0.5,
        background='scatter',
        num_pix=50, 
        filter_type='gauss',
        sigma_val=1,
        radius_val=3,
        scatter_s=0.2,
        density=2,
        seed_val=0,
        num_legend_per_col=20,
        dpi_val=1000,
        out_path =  outdir + name + '_dpt_stream_scatter.svg'
        )

data.plt.plot_vec(
        data.position[:, 0] - data.position[:, 0].min(),
        data.position[:, 1] - data.position[:, 1].min(),
        data.cells['leiden'].to_numpy(),
        data.tl.result['dpt_pseudotime'],
        type='vec',
        background='field',
        num_pix=50,
        filter_type='gauss',
        sigma_val=2,
        radius_val=1,
        seed_val=0,
        dpi_val=1000,
        out_path =  outdir + name + '_dpt_vec_field.svg'
        )

data.plt.plot_vec(
        data.position[:, 0] - data.position[:, 0].min(),
        data.position[:, 1] - data.position[:, 1].min(),
        data.cells['leiden'].to_numpy(),
        data.tl.result['dpt_pseudotime'],
        type='vec',
        background='scatter',
        num_pix=50,
        filter_type='gauss',
        sigma_val=2,
        radius_val=1,
        scatter_s=0.2,
        seed_val=0,
        dpi_val=1000,
        out_path =  outdir + name + '_dpt_vec_scatter.svg'
        )

data.plt.plot_time_scatter(
    group='leiden', 
    plotting_scale_width=400,
    out_path =  outdir + name + '_dpt_time_scatter.pdf'
)

from stereo.tools import generate_loom

gtf_file = '/data/work/input/SL4.0_MT_CP_34900genes.gtf'

# generate loom file
loom_data = generate_loom(
                gef_path=bgef_file,
                gtf_path=gtf_file,
                bin_type='bins',
                bin_size=20,
                out_dir=outdir
                )

import dynamo as dyn

# set visualization mode
dyn.configuration.set_figure_params('dynamo', background='white')

# read data
adata = dyn.read_loom(loom_data)
dyn.pp.recipe_monocle(
            adata,
            num_dim=30,
            keep_filtered_genes = True
            )

dyn.tl.dynamics(adata, model='stochastic', cores=60)
dyn.tl.moments(adata)
dyn.tl.reduceDimension(adata)

adata.obs['x'] = list(map(lambda x: float(x.split("_")[0]) ,list(adata.obs.index)))
adata.obs['y'] = list(map(lambda x: float(x.split("_")[1]) ,list(adata.obs.index)))

adata.obsm['spatial'] = adata.obs[['x', 'y']].values.astype(float)
adata.obsm['X_spatial'] = adata.obs[['x', 'y']].values.astype(float)

dyn.tl.cell_velocities(
            adata, 
            method='fp', 
            basis='X_spatial', 
            enforce=True,  
            transition_genes = list(adata.var_names[adata.var.use_for_pca])
            )

dyn.tl.cell_wise_confidence(adata)
dyn.tl.leiden(adata, result_key='spatial_leiden_res')

dyn.pl.streamline_plot(
            adata, 
            color = 'spatial_leiden_res', 
            basis='X_spatial',
            quiver_length=6, 
            quiver_size=6,  
            show_arrowed_spines=True,
            show_legend=False,
            figsize=(10, 10),
            theme='viridis',
            save_show_or_return='save',
            save_kwargs= {'path': outdir, 'prefix': name, 'dpi': 300, 'ext': 'svg', 'transparent': True, 'close': True, 'verbose': True} 
            )